# LAB 3.1: Supervised Learning

**Temat:** Regresja i Klasyfikacja

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

np.random.seed(42)
print('✓ Biblioteki załadowane')

## CZĘŚĆ 1: REGRESJA

### California Housing Dataset

In [ ]:
# TODO: Wczytaj California Housing dataset
# Użyj funkcji fetch_california_housing()
housing = None # TODO
X = None # TODO
y = None # TODO
feature_names = None # TODO

# print(f'Shape: {X.shape}')

In [ ]:
# EDA - Analiza korelacji
if X is not None:
    housing_df = pd.DataFrame(X, columns=feature_names)
    housing_df['target'] = y
    plt.figure(figsize=(10, 8))
    sns.heatmap(housing_df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.show()

In [ ]:
# TODO: Podziel dane na zbiór treningowy i testowy (80% train, 20% test)
X_train, X_test, y_train, y_test = None, None, None, None # TODO

# TODO: Przeskaluj dane używając StandardScaler
scaler = StandardScaler()
X_train_scaled = None # TODO: fit_transform na train
X_test_scaled = None # TODO: transform na test

### Zadanie 1.1: Linear Regression

In [ ]:
# TODO: Zainicjalizuj i wytrenuj Linear Regression
lr = None # TODO
# TODO: lr.fit(...)

# TODO: Dokonaj predykcji na zbiorze testowym
y_pred_test = None # TODO

# Metryki
if y_pred_test is not None:
    print(f'Test MSE: {mean_squared_error(y_test, y_pred_test):.4f}')
    print(f'Test R²: {r2_score(y_test, y_pred_test):.4f}')

### Zadanie 1.2: Ridge Regression (L2)

In [ ]:
# TODO: Przetestuj różne wartości alpha w pętli
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
results = []

for alpha in alphas:
    # TODO: Trenuj Ridge Regression z danym alpha
    # ridge = ...
    # ridge.fit(...)
    # score = ...
    score = 0 # Placeholder
    results.append({'alpha': alpha, 'r2': score})

results_df = pd.DataFrame(results)
print(results_df)

### Zadanie 1.3: Lasso Regression (L1)

In [ ]:
# TODO: Zastosuj Lasso Regression (alpha=0.1) i sprawdź, które cechy zostały wyzerowane
lasso = None # TODO

# if lasso:
#     print(f'Features z coefficient = 0: {np.sum(lasso.coef_ == 0)}')

## CZĘŚĆ 2: KLASYFIKACJA

### Breast Cancer Dataset

In [ ]:
# TODO: Wczytaj dane Breast Cancer
breast_cancer = load_breast_cancer()
X_bc = breast_cancer.data
y_bc = breast_cancer.target

# TODO: Podziel na train/test (stratify=y_bc) i przeskaluj dane
X_train_bc_scaled = None # TODO
X_test_bc_scaled = None # TODO
y_train_bc = None
y_test_bc = None

### Zadanie 2.1: Porównanie Modeli

In [ ]:
# TODO: Zdefiniuj słownik z modelami: LogisticRegression, DecisionTree, RandomForest, SVM, KNN
models = {
    'Logistic Regression': None, # TODO
    'Decision Tree': None,
    'Random Forest': None,
    'SVM': None,
    'KNN': None
}

# Pętla trenująca i oceniająca modele
for name, model in models.items():
    if model is not None:
        # TODO: fit i predict
        # model.fit(...)
        # y_pred = ...
        pass

### Zadanie 2.2: Analiza Najlepszego Modelu

In [ ]:
# TODO: Wybierz najlepszy model (np. Random Forest) i wyświetl macierz pomyłek
best_model = None
# best_model.fit(...)

# TODO: Classification Report

# TODO: Feature Importance (jeśli model na to pozwala)